In [98]:
import cv2
import numpy as np
import os
import pandas as pd
import datetime
import random

In [99]:
video_path = '/mnt/teams/TM_Lab/Arjun Bhaskaran/Social interaction project/videos/'
processed_data_path = 'successful_trials_by_day'
reference_path = ['marker_reference/' + dir for dir in os.listdir('marker_reference')]

In [100]:
def process_time_string(time_string):
    time_string = str(time_string)
    if time_string == 'nan':
        return None
    if len(time_string) > 14:
        millis = float(time_string[14:])
    else:
        millis = 0
    secs = int(time_string[12:14])
    mins = int(time_string[10:12])
    hours = int(time_string[8:10])
    day = int(time_string[6:8])
    month = int(time_string[4:6])
    year = int(time_string[0:4])
    # divide millis until there is nothing to the right of the decimal point
    while millis > 1:
        millis /= 10
    return datetime.datetime(year, month, day, hours, mins, secs, int(millis * 1000))


# Using OpenCV, takes a reference image of the interaction area and the current frame
# using the reference image finds a perspective transform to transform and crop the current frame to the reference frame using SIFT algorithm
def find_area_and_transform(frame, reference):
    # create SIFT object
    sift = cv2.SIFT_create()
    # find keypoints and descriptors for reference and current frame
    kp1, des1 = sift.detectAndCompute(reference,None)
    kp2, des2 = sift.detectAndCompute(frame,None)
    # create BFMatcher object
    matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
    knn_matches = matcher.knnMatch(des1, des2, 2)
    good = []
    for m,n in knn_matches:
        if m.distance < 0.75*n.distance:
            good.append(m)

    #-- Localize the object
    obj = np.empty((len(good),2), dtype=np.float32)
    scene = np.empty((len(good),2), dtype=np.float32)
    for i in range(len(good)):
        #-- Get the keypoints from the good matches
        obj[i,0] = kp1[good[i].queryIdx].pt[0]
        obj[i,1] = kp1[good[i].queryIdx].pt[1]
        scene[i,0] = kp2[good[i].trainIdx].pt[0]
        scene[i,1] = kp2[good[i].trainIdx].pt[1]
    # find the perspective transform
    try:
        M, mask = cv2.findHomography(scene, obj, cv2.RANSAC,5.0)
        return M, len(good)
    except:
        return None, 0


def scrub_video_seconds(vid_capture, seconds):
    fps = vid_capture.get(cv2.CAP_PROP_FPS)
    frame_count = int(vid_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    current_pos_seconds = frame_count / fps
    time_to_skip = current_pos_seconds - seconds
    for i in range(int(time_to_skip * fps)):
        vid_capture.read()
    return vid_capture

# iterate through list of paths and get the best matching SIFT transformation
def find_matching_transformation(scene , paths):
    references_images = [cv2.imread(path) for path in paths]
    transformations = []
    num_matches = []
    for reference in references_images:
        transformation, num_match = find_area_and_transform(scene, reference)
        transformations.append(transformation)
        num_matches.append(num_match)
    return transformations[num_matches.index(max(num_matches))], max(num_matches)

def show_successfull_trials(video, df, offset):
     # iterate through df and for each successfull trial get the start time in seconds
    reference_image = cv2.imread(reference_path[0])
    transformation, num_match = find_matching_transformation(video.read()[1], reference_path)
    for index, row in df.iterrows():
        if row['successful_trials']:
            transformation_new, num_match_new = find_matching_transformation(video.read()[1], reference_path)
            if num_match_new > num_match or num_match_new > 30:
                transformation = transformation_new
                num_match = num_match_new
            start_time_str = row['time_since_start'].split(' ')[2]
            start_time = start_time_str.split(':')
            start_time = float(start_time[0])*3600 + float(start_time[1])*60 + float(start_time[2]) + offset
            end_time = start_time + 15
            # set the video to the start time
            fps = video.get(cv2.CAP_PROP_FPS)
            current_time = float(video.get(cv2.CAP_PROP_POS_FRAMES)) / fps
            time_offset = start_time - current_time
            for i in range(int(time_offset * video.get(cv2.CAP_PROP_FPS))):
                video.read()

            # find the frames where mouse R and L break the beam
            R_time_offset = row['R']
            L_time_offset = row['L']
            
            # find the mice names
            name1 = row['names'][:2]
            name2 = row['names'][2:]
            # display the video until the end time
            
            while True:
                success, image = video.read()
                if success == False or float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps > end_time:
                    break
                # add a countdown to the video 
                time_since_start_seconds = float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps
                time_since_start = str(datetime.timedelta(seconds=time_since_start_seconds))
                transformed = cv2.warpPerspective(image, transformation, (reference_image.shape[1], reference_image.shape[0]))

                cv2.putText(image, start_time_str + ' ' + time_since_start, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                # display the mice names
                cv2.putText(image, 'trial: ' + str(index), (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, name1 + ' ' + name2, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(image, str(df['cue_times'])[:18], (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.imshow('frame', image)
                cv2.imshow('transformed', transformed)

                if cv2.waitKey(12) & 0xFF == ord('q'):
                    break

def write_successful_trials(video, df, offset, path):
    # iterate through df and for each successfull trial get the start time in seconds
    reference_image = cv2.imread(reference_path[0])
    transformation, num_match = find_matching_transformation(video.read()[1], reference_path)
    # create video writer
    fps = video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    for index, row in df.iterrows():
        if row['successful_trials']:
            transformation_new, num_match_new = find_matching_transformation(video.read()[1], reference_path)
            if num_match_new > num_match or num_match_new > 30:
                transformation = transformation_new
                num_match = num_match_new
            start_time_str = row['time_since_start'].split(' ')[2]
            start_time = start_time_str.split(':')
            start_time = float(start_time[0])*3600 + float(start_time[1])*60 + float(start_time[2]) + offset
            end_time = start_time + 15
            # set the video to the start time
            fps = video.get(cv2.CAP_PROP_FPS)
            current_time = float(video.get(cv2.CAP_PROP_POS_FRAMES)) / fps
            time_offset = start_time - current_time
            for i in range(int(time_offset * video.get(cv2.CAP_PROP_FPS))):
                video.read()

            # find the mice names
            name1 = row['names'][:2]
            name2 = row['names'][2:]
            # write the video to a new file until the end time
            # check if folder exists, if not create it
            if not os.path.exists(path):
                os.makedirs(path)
            out = cv2.VideoWriter(path + '_trial_' +str(index) + '.mp4', fourcc, fps, (reference_image.shape[1], reference_image.shape[0]))

            while True:
                success, image = video.read()
                if success == False or float(video.get(cv2.CAP_PROP_POS_FRAMES))/fps > end_time:
                    break
                # add a countdown to the video 
                transformed = cv2.warpPerspective(image, transformation, (reference_image.shape[1], reference_image.shape[0]))
                out.write(transformed)
                if cv2.waitKey(12) & 0xFF == ord('q'):
                    break
            out.release()


In [101]:
# get list of video names from the folder of processed CSVs
csv_times = [ process_time_string(name.split('_')[0]) for name in os.listdir(processed_data_path)]
csv_paths = [name for name in os.listdir(processed_data_path)]
# iterate through each csv and find the closest video name
video_times = [process_time_string(name.split('_')[0]) for name in os.listdir(video_path)]
video_paths = [name for name in os.listdir(video_path)]
csv_vid_match = dict()
for csv_time in csv_times:
    closest_time = video_times[0]
    closest_path = video_paths[0]
    for i, video_time in enumerate(video_times):
        if abs((csv_time - video_time).total_seconds()) <  abs((csv_time - closest_time).total_seconds()):
            closest_time = video_time
            closest_path = video_paths[i]
    total_delta = (csv_time - closest_time).total_seconds()
    if total_delta < 10:
        csv_vid_match[csv_paths[csv_times.index(csv_time)]] = {
            'video_name': closest_path,
            'delta_seconds': total_delta
        }

significant_trials = []
for csv_name in csv_vid_match.keys():
    df = pd.read_csv(processed_data_path + '/' + csv_name)
    if len(df['successful_trials']) >= 10:
        significant_trials.append(csv_name)
significant_trials.sort()
significant_trials.reverse()
# randoize order
random.shuffle(significant_trials)


    

In [102]:
# iterate throught each video and load the corresponding video and csv and then display the video 

for csv_name in significant_trials:
    # check if folder for trial exists, if so skip
    if os.path.exists('/mnt/teams/Tsuchitori/social_interaction_trials/' + csv_name.split('_')[0] + '/'):
        continue
    video = cv2.VideoCapture(video_path + csv_vid_match[csv_name]['video_name'])
    df = pd.read_csv(processed_data_path + '/' + csv_name)
    if len(df['successful_trials']) < 10:
        continue
    write_successful_trials(video, df, csv_vid_match[csv_name]['delta_seconds'], '/mnt/teams/Tsuchitori/social_interaction_trials/' + csv_name.split('_')[0] + '/')


In [103]:
# for test_path in significant_trials:
#     test_video = cv2.VideoCapture(video_path + csv_vid_match[test_path]['video_name'])
#     test_video = scrub_video_seconds(test_video, 10)
#     # get single frame
#     success, image = test_video.read()
#     # boost contrast
#     cv2.imshow('original', image)
#     # create slider to adjust contrast
#     def nothing(x):
#         pass
#     # create bars to crop image
#     cv2.createTrackbar('x1', 'original', 0, 1000, nothing)
#     cv2.createTrackbar('y1', 'original', 0, 1000, nothing)
#     cv2.createTrackbar('x2', 'original', 1, 1000, nothing)
#     cv2.createTrackbar('y2', 'original', 1, 1000, nothing)
#     cv2.setTrackbarPos('x2', 'original', 200)
#     cv2.setTrackbarPos('y2', 'original', 200)
#     while True:
#         # crop newimage 
#         x1 = cv2.getTrackbarPos('x1', 'original')
#         y1 = cv2.getTrackbarPos('y1', 'original')
#         x2 = cv2.getTrackbarPos('x2', 'original')
#         y2 = cv2.getTrackbarPos('y2', 'original')
#         new_image = image[y1:y2, x1:x2]
#         cv2.imshow('frame', new_image)
#         cv2.imshow('original', image)

#         if cv2.waitKey(12) & 0xFF == ord('q'):
#             break    
#     cv2.destroyAllWindows()
#     # save the image to a file
#     # display the image
#     cv2.imwrite('marker_reference/interaction_area_' + test_path.split('.')[0] + '.jpg', new_image)

KeyboardInterrupt: 